In [12]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model

In [102]:
properties = pd.read_csv('properties_2016.csv')
train = pd.read_csv('train_2016.csv')
sampleSub = pd.read_csv('sample_submission.csv')

In [103]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


In [104]:
properties.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [105]:
train.head()

,parcelid,logerror,transactiondate
0,11016594,0.0276,2016-01-01
1,14366692,-0.1684,2016-01-01
2,12098116,-0.0040,2016-01-01
3,12643413,0.0218,2016-01-02
4,14432541,-0.0050,2016-01-02


# Filling in NA Values

In [106]:
airConditionMode = properties['airconditioningtypeid'].value_counts().argmax()
properties['airconditioningtypeid'] = properties['airconditioningtypeid'].fillna(airConditionMode)

architectureMode = properties['architecturalstyletypeid'].value_counts().argmax()
properties['architecturalstyletypeid'] = properties['architecturalstyletypeid'].fillna(architectureMode)

basementSqFeetAverage = properties['basementsqft'].mean()
properties['basementsqft'] = properties['basementsqft'].fillna(basementSqFeetAverage)

bathroomCntMode = properties['bathroomcnt'].value_counts().argmax()
properties['bathroomcnt'] = properties['bathroomcnt'].fillna(bathroomCntMode)

bedroomCntMode = properties['bedroomcnt'].value_counts().argmax()
properties['bedroomcnt'] = properties['bedroomcnt'].fillna(bedroomCntMode)

buildingClassType = properties['buildingclasstypeid'].value_counts().argmax()
properties['buildingclasstypeid'] = properties['buildingclasstypeid'].fillna(buildingClassType)

buildingQualityType = properties['buildingqualitytypeid'].value_counts().argmax()
properties['buildingqualitytypeid'] = properties['buildingqualitytypeid'].fillna(buildingQualityType)

calculatedBathnBedroom = properties['calculatedbathnbr'].value_counts().argmax()
properties['calculatedbathnbr'] = properties['calculatedbathnbr'].fillna(calculatedBathnBedroom)

# Making deck type a binary label
properties['decktypeid'] = properties['decktypeid'].fillna(0) 
properties['decktypeid'] = properties['decktypeid'].replace(66,1)

floor1SqFeetAverage = properties['finishedfloor1squarefeet'].mean()
properties['finishedfloor1squarefeet'] = properties['finishedfloor1squarefeet'].fillna(floor1SqFeetAverage)

calculatedSqFeetAverage = properties['calculatedfinishedsquarefeet'].mean()
properties['calculatedfinishedsquarefeet'] = properties['calculatedfinishedsquarefeet'].fillna(calculatedSqFeetAverage)

finishedSqFeet6 = properties['finishedsquarefeet6'].mean()
properties['finishedsquarefeet6'] = properties['finishedsquarefeet6'].fillna(finishedSqFeet6)

finishedSqFeet12 = properties['finishedsquarefeet12'].mean()
properties['finishedsquarefeet12'] = properties['finishedsquarefeet12'].fillna(finishedSqFeet12)

finishedSqFeet13 = properties['finishedsquarefeet13'].mean()
properties['finishedsquarefeet13'] = properties['finishedsquarefeet13'].fillna(finishedSqFeet13)

finishedSqFeet15 = properties['finishedsquarefeet15'].mean()
properties['finishedsquarefeet15'] = properties['finishedsquarefeet15'].fillna(finishedSqFeet15)

finishedSqFeet50 = properties['finishedsquarefeet50'].mean()
properties['finishedsquarefeet50'] = properties['finishedsquarefeet50'].fillna(finishedSqFeet50)

fips = properties['fips'].value_counts().argmax()
properties['fips'] = properties['fips'].fillna(fips)

# Making fireplace count\ a binary label
properties['fireplacecnt'] = properties['fireplacecnt'].replace([2,3,4,5,6,7,8,9],1)
properties['fireplacecnt'] = properties['fireplacecnt'].fillna(0) 

fullCntBathMode = properties['fullbathcnt'].value_counts().argmax()
properties['fullbathcnt'] = properties['fullbathcnt'].fillna(fullCntBathMode)

garageCntMode = properties['garagecarcnt'].value_counts().argmax()
properties['garagecarcnt'] = properties['garagecarcnt'].fillna(garageCntMode)

garageSqFeetMode = properties['garagetotalsqft'].value_counts().argmax()
properties['garagetotalsqft'] = properties['garagetotalsqft'].fillna(garageSqFeetMode)

# Making hot tub a binary label
properties['hashottuborspa'] = properties['hashottuborspa'].replace(True,1)
properties['hashottuborspa'] = properties['hashottuborspa'].fillna(0)

heatingMode = properties['heatingorsystemtypeid'].value_counts().argmax()
properties['heatingorsystemtypeid'] = properties['heatingorsystemtypeid'].fillna(heatingMode)

properties = properties.drop('latitude', axis=1)
properties = properties.drop('longitude', axis=1)

lotSizeMode = properties['lotsizesquarefeet'].value_counts().argmax()
properties['lotsizesquarefeet'] = properties['lotsizesquarefeet'].fillna(lotSizeMode)

# Making pool a binary label
properties['poolcnt'] = properties['poolcnt'].fillna(0)

properties['poolsizesum'] = properties['poolsizesum'].fillna(0)

# These properties show through with the previous features
properties = properties.drop('pooltypeid10', axis=1)
properties = properties.drop('pooltypeid2', axis=1)
properties = properties.drop('pooltypeid7', axis=1)

# Why would these even impact the price (but idk, maybe they're important)?
properties = properties.drop('propertycountylandusecode', axis=1)
properties = properties.drop('propertylandusetypeid', axis=1)
properties = properties.drop('propertyzoningdesc', axis=1)
properties = properties.drop('rawcensustractandblock', axis=1)
properties = properties.drop('censustractandblock', axis=1)

properties['regionidcounty'] = properties['regionidcounty'].replace([3101, 1286, 2061],[0,1,2])

# No idea how to handle these features
properties = properties.drop('regionidcity', axis=1)
properties = properties.drop('regionidzip', axis=1)
properties = properties.drop('regionidneighborhood', axis=1)

# Don't bedroom and bathroom counts already do this?
properties = properties.drop('roomcnt', axis=1)
properties = properties.drop('threequarterbathnbr', axis=1)

# Making deck type a binary label
properties['storytypeid'] = properties['storytypeid'].fillna(0) 
properties['storytypeid'] = properties['storytypeid'].replace(7,1)

# Only has like a couple thousand non NA values, so not worth
properties = properties.drop('typeconstructiontypeid', axis=1)

unitMode = properties['unitcnt'].value_counts().argmax()
properties['unitcnt'] = properties['unitcnt'].fillna(unitMode)

yardSqFt17 = properties['yardbuildingsqft17'].mean()
properties['yardbuildingsqft17'] = properties['yardbuildingsqft17'].fillna(yardSqFt17)

yardSqFt26 = properties['yardbuildingsqft26'].mean()
properties['yardbuildingsqft26'] = properties['yardbuildingsqft26'].fillna(yardSqFt26)

yearBuilt = properties['yearbuilt'].mean()
properties['yearbuilt'] = properties['yearbuilt'].fillna(yearBuilt)

properties['numberofstories'] = properties['numberofstories'].fillna(0)

# Fireplace count already does this
properties = properties.drop('fireplaceflag', axis=1)

structureTax = properties['structuretaxvaluedollarcnt'].mean()
properties['structuretaxvaluedollarcnt'] = properties['structuretaxvaluedollarcnt'].fillna(structureTax)

landTax = properties['landtaxvaluedollarcnt'].mean()
properties['landtaxvaluedollarcnt'] = properties['landtaxvaluedollarcnt'].fillna(landTax)

tax = properties['taxamount'].mean()
properties['taxamount'] = properties['taxamount'].fillna(tax)

# Tax amount already does this
properties = properties.drop('taxvaluedollarcnt', axis=1)

# Idk, I don't wanna deal with these
properties = properties.drop('assessmentyear', axis=1)
properties = properties.drop('taxdelinquencyflag', axis=1)
properties = properties.drop('taxdelinquencyyear', axis=1)

In [107]:
properties.columns

Index([u'parcelid', u'airconditioningtypeid', u'architecturalstyletypeid',
       u'basementsqft', u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'lotsizesquarefeet', u'poolcnt',
       u'poolsizesum', u'regionidcounty', u'storytypeid', u'unitcnt',
       u'yardbuildingsqft17', u'yardbuildingsqft26', u'yearbuilt',
       u'numberofstories', u'structuretaxvaluedollarcnt',
       u'landtaxvaluedollarcnt', u'taxamount'],
      dtype='object')

In [108]:
cleanedPropertyData = pd.np.array(properties)
cleanedPropertyData.shape

(2985217, 37)

In [109]:
cleanedPropertyData.nbytes

883624232